In [ ]:
'''
씽유 공모전 과학/공학 항목을 선택해도 문학 공모전 중 해양, 생태와 같은 키워드가 있을 경우 포함되는 경우가 있다.
키워드 포함 시 제외와 같은 방법으로 해결해야 할 것으로 보인다.
'''

In [ ]:
def browser():
    # 씽유 홈페이지 

    url ='https://thinkyou.co.kr/contest/sector.asp'

    browser = Chrome('./chromedriver')

    delay=3
    browser.implicitly_wait(delay)

    browser.get(url)

    browser.maximize_window()

    body = browser.find_element_by_tag_name('body')

    try :
        # 과학 / 공학 * 대학(원)생 카테고리

        browser.find_elements_by_xpath('//*[@id="searchFrm"]/div/dl[1]/dd/p[6]/label/span')[0].click() #과학/공학
        browser.find_elements_by_xpath('//*[@id="searchFrm"]/div/dl[2]/dd/p[1]/label/span')[0].click() #대학(원)생
    except:
        pass

    SCROLL_PAUSE_TIME = 0.5
    # 스크롤 내려가게
    while True:
        last_height = browser.execute_script('return document.documentElement.scrollHeight')

        for i in range(3):
            body.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)
        new_height = browser.execute_script('return document.documentElement.scrollHeight')
        if new_height == last_height:
            break;

    page = browser.page_source
    soup = BeautifulSoup(page, 'lxml')
    return soup

In [ ]:
def crawling():
    soup = browser()

    len_day = 20

    links_bef = []
    titles_bef = []
    dday_bef = []
    inst_bef = []
    start_bef = []
    end_bef = []

    links_aft = []
    titles_aft = []
    inst_aft = []

    for i in range(len_day):
        t = soup.select(' .title > a > dl > dt ')[i].text
        fin = soup.select(' .statNew > p ')[i].text
        
        
        # 마감된 공모전 가져오고 싶은 경우
        if fin == '마감':
            base_url = 'https://thinkyou.co.kr'
            titles_aft.append(soup.select(' .title > a > dl > dt ')[i].text)
            inst_aft.append(soup.select(' .title > a > dl > dd ')[i].text.split(':')[1][1:])

            links_aft.append(base_url + soup.select(' .title > a')[i]['href'][2:])
        else:
            stand = soup.select(' .statNew')[i].text.split('D')[1]
            
            
            # D-Day인 경우
            if stand == '-day':
                num = 0
                base_url = 'https://thinkyou.co.kr'
                titles_bef.append(soup.select(' .title > a > dl > dt ')[i].text)
                inst_bef.append(soup.select(' .title > a > dl > dd ')[i].text.split(':')[1][1:])
                dday_bef.append(num)
                links_bef.append(base_url + soup.select(' .title > a')[i]['href'][2:])
                a = i * 2
                st = soup.select(' .etc')[a].text[:8]
                it = st.split('-')
                ti = datetime.datetime(int(it[0]), int(it[1]), int(it[2]))
                st_3 = ti.strftime('%Y-%m-%d')

                start_bef.append(st_3[2:])
                
                ed = soup.select(' .etc')[a].text[11:]
                it_e = ed.split('-')
                ti_e = datetime.datetime(int(it_e[0]), int(it_e[1]), int(it_e[2]))
                ed_3 =ti_e.strftime('%Y-%m-%d')
                end_bef.append(ed_3[2:])

            else:
                # D-숫자 인경우

                base_url = 'https://thinkyou.co.kr'
                titles_bef.append(soup.select(' .title > a > dl > dt ')[i].text)
                inst_bef.append(soup.select(' .title > a > dl > dd ')[i].text.split(':')[1][1:])
                dday_bef.append(soup.select(' .statNew')[i].text.split('-')[1])
                links_bef.append(base_url + soup.select(' .title > a')[i]['href'][2:])
                a = i * 2
                st = soup.select(' .etc')[a].text[:8]
                it = st.split('-')
                ti = datetime.datetime(int(it[0]), int(it[1]), int(it[2]))
                st_3 = ti.strftime('%Y-%m-%d')

                start_bef.append(st_3[2:])
                
                ed = soup.select(' .etc')[a].text[11:]
                it_e = ed.split('-')
                ti_e = datetime.datetime(int(it_e[0]), int(it_e[1]), int(it_e[2]))
                ed_3 =ti_e.strftime('%Y-%m-%d')
                end_bef.append(ed_3[2:])
                
    print(inst_bef)

    tabl_data_bef = {'title': titles_bef, 'notice': start_bef, 'deadline': end_bef, 
                     'dday': dday_bef, 'sponsor': inst_bef, 'title2': titles_bef, 'link': links_bef}
    print(tabl_data_bef)

    df_bef = pd.DataFrame(tabl_data_bef, columns=['type', 'qualification', 'title', 
                                                  'notice', 'deadline', 'dday', 'sponsor', 'title2', 'link'])
    
    df_bef['type'] = '2 공모전'
    
    
    df_bef['qualification'] = '대학(원)생'


    return df_bef